In [ ]:
import pandas as pd

In [3]:
df = pd.read_csv('data/agustin_video_EEG.csv')

In [16]:
df.head(10)

,channel,par_id,class,stimulus,1,2,3,4,5,6,...,5115,5116,5117,5118,5119,5120,5121,5122,5123,5124
0,1,AF3,1,BDay_Joy,Pleasant,8,3,-0.625861,-0.209014,0.194518,...,-6.579655,-6.892154,-7.117153,-7.324761,-7.460088,-7.589979,-7.692696,-7.763348,-7.800576,-7.818239
1,2,AF4,1,BDay_Joy,Pleasant,8,3,0.475098,0.641946,0.795478,...,-6.978693,-7.166192,-7.203691,-7.130050,-7.015377,-6.864019,-6.716737,-6.662388,-6.668366,-6.779779
2,3,AF7,1,BDay_Joy,Pleasant,8,3,-1.091605,-0.956008,-0.771225,...,-6.107901,-6.076651,-6.082901,-6.103009,-6.207085,-6.399476,-6.689693,-7.041594,-7.485071,-7.971484
3,4,AF8,1,BDay_Joy,Pleasant,8,3,-1.333793,-1.323195,-1.294662,...,-9.256332,-9.443831,-9.450082,-9.345191,-9.168017,-8.954159,-8.744377,-8.565029,-8.446008,-8.401171
4,5,AFF1H,1,BDay_Joy,Pleasant,8,3,-0.693770,-0.495672,-0.373390,...,-6.397564,-6.460063,-6.466313,-6.455172,-6.465498,-6.532890,-6.698106,-6.925008,-7.274735,-7.729898
5,6,AFF2H,1,BDay_Joy,Pleasant,8,3,-0.651702,-0.359855,-0.050073,...,-5.792997,-5.917997,-5.955497,-5.881856,-5.767182,-5.678324,-5.624791,-5.632943,-5.763922,-6.000334
6,7,AFF5H,1,BDay_Joy,Pleasant,8,3,-0.336198,-0.263100,-0.172068,...,-6.321242,-6.414991,-6.452492,-6.503850,-6.576676,-6.737817,-6.965534,-7.286185,-7.698413,-8.153575
7,8,AFF6H,1,BDay_Joy,Pleasant,8,3,-0.867447,-0.856849,-0.922067,...,-2.914998,-3.289997,-3.577497,-3.816355,-4.014180,-4.175322,-4.371788,-4.567440,-4.792168,-5.028581
8,9,AFP1,1,BDay_Joy,Pleasant,8,3,0.208873,0.406971,0.591753,...,-5.963670,-5.869920,-5.719921,-5.583780,-5.437856,-5.380247,-5.420465,-5.553616,-5.809595,-6.171007
9,10,AFP2,1,BDay_Joy,Pleasant,8,3,-0.961197,-0.888099,-0.797067,...,-6.008742,-6.164992,-6.171241,-6.097601,-5.982927,-5.894069,-5.809286,-5.848688,-5.979666,-6.216078


In [5]:
downsampled_df = pd.DataFrame()
downsampled_df = df.iloc[:, :7].copy()
for i in range(7, df.shape[1], 4):
    downsampled_df[i] = df.iloc[:, i:i+4].mean(axis=1)

In [6]:
for index, row in downsampled_df.iterrows():
    valence, arousal = row["valence"], row["arousal"]
    if valence < 4.5 and arousal < 4.5: # LVLA
        cls = 0
    elif valence < 4.5:                 # LVHA
        cls = 1
    elif arousal < 4.5:                 # HVLA
        cls = 2
    else:                               # HVHA
        cls = 3
    downsampled_df.at[index, "class"] = cls

/tmp/ipykernel_2797898/3907811484.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  downsampled_df.at[index, "class"] = cls


In [7]:
downsampled_df = downsampled_df.drop(columns=["Unnamed: 0", "Stim_cat", "valence", "arousal"])

In [8]:
cols = list(downsampled_df.columns)
cols.insert(2, cols.pop(-1))
downsampled_df = downsampled_df[cols]

In [14]:
downsampled_df.rename(columns={"EEG_channel_name": "channel"}, inplace=True)
downsampled_df.rename(columns={"Stim_name": "stimulus"}, inplace=True)
new_columns = ["channel", "par_id", "class", "stimulus"]  # Keep these columns unchanged
new_columns += [str(i) for i in range(1, len(downsampled_df.columns) - 4 + 1)]
downsampled_df.columns = new_columns

In [15]:
downsampled_df.head(-128)

,channel,par_id,class,stimulus,1,2,3,4,5,6,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281
0,AF3,1,2.0,BDay_Joy,-0.020224,0.881066,-0.698144,-3.549089,-4.491411,-2.933262,...,2.278413,5.953610,7.393146,5.182416,0.782153,-3.341413,-5.935158,-7.198539,-7.711650,-7.818239
1,AF4,1,2.0,BDay_Joy,0.697924,0.661715,-1.050306,-4.135627,-7.124820,-8.316665,...,5.379369,5.749885,5.947235,5.041191,1.883112,-2.740453,-6.240446,-7.128828,-6.727877,-6.779779
2,AF7,1,2.0,BDay_Joy,-0.860967,-0.654989,-2.132636,-4.303896,-4.738406,-3.531818,...,1.023608,3.081621,3.349283,1.185428,-2.347649,-5.127468,-6.080591,-6.117411,-6.903959,-7.971484
3,AF8,1,2.0,BDay_Joy,-1.306279,-1.412801,-2.468573,-4.913269,-7.996212,-9.813057,...,4.226727,3.667557,1.786786,0.154180,-2.105462,-5.588404,-8.596211,-9.351780,-8.677393,-8.401171
4,AFF1H,1,2.0,BDay_Joy,-0.470944,-0.772778,-2.945736,-5.663869,-6.832753,-5.993352,...,2.976128,6.331014,8.176798,6.442630,1.643930,-3.339009,-5.995254,-6.461761,-6.857685,-7.729898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429819,TP9,50,3.0,Zipline,5.088190,5.630052,4.867451,3.903346,2.368119,-0.416517,...,-4.282444,-3.986454,-5.850248,-9.345377,-11.500886,-10.307492,-6.857690,-3.769915,-2.436265,-2.203144
429820,TPP7H,50,3.0,Zipline,3.775693,3.692556,3.429953,3.801783,4.008741,2.591290,...,-1.766824,-1.509896,-2.545566,-4.040699,-4.407149,-3.299692,-1.709262,-0.746484,-0.772206,-1.195333
429821,TPP8H,50,3.0,Zipline,1.867042,4.674525,4.763485,1.768133,-0.806154,0.073266,...,-0.433292,-2.324798,-4.352654,-6.339974,-7.237672,-5.770842,-2.399164,0.524548,1.155074,0.481948
429822,TTP7H,50,3.0,Zipline,2.660911,1.569964,0.338613,0.499506,0.956463,-0.335988,...,-2.217544,-1.773117,-2.832225,-4.780481,-5.521930,-4.312912,-2.644356,-2.158140,-2.644798,-2.896051


In [17]:
downsampled_df.to_csv("data/preprocessed_video.csv", index=False)